In [1]:
import numpy as np
import pandas as pd
from transformers import AutoImageProcessor, ResNetModel
import torch
from datasets import load_dataset, Image
from sklearn.metrics import precision_recall_fscore_support
# from PIL import Image

In [2]:
# importing the data
datafolder = '../../data/hateful_memes/'
train = datafolder+'train_with_features.csv'
test = datafolder+'test_with_features.csv'
dev = datafolder+'dev_with_features.csv'

In [3]:
dataset = load_dataset('csv',data_files={"train": train, "validation": dev, "test": test},  num_proc=8).cast_column("img", Image(decode=False))

Found cached dataset csv (C:/Users/Hisha/.cache/huggingface/datasets/csv/default-a188762e54a0d8f3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from PIL import Image
# Initialize image processor and ResNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, otherwise use CPU
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
model = ResNetModel.from_pretrained("microsoft/resnet-50").to(device)
def get_image_vectors(dataset, split):
    # Initialize list to store outputs
    outputs_list = []

    # Iterate through all images in the dataset
    for i in range(len(dataset[split]['img'])):

        # Open image using PIL
        image = Image.open(datafolder+dataset[split]['img'][i]['path'])
        
        # Check image format and convert if necessary
        if image.mode != "RGB":
            image = image.convert("RGB")

        # Check image size and resize if necessary
        if image.size != (224, 224):
            image = image.resize((224, 224))
            
        # Preprocess image using the image processor
        inputs = image_processor(image, return_tensors="pt").to(device)

        # Pass image through the ResNet model
        with torch.no_grad():
            outputs = model(**inputs)

        # Append the last hidden state to the outputs list
        outputs_list.append(outputs.last_hidden_state.to(device))
    return outputs_list
        # outputs_list.append(outputs.last_hidden_state.numpy().flatten().tolist())        

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [ ]:
# train_img = get_image_vectors(dataset, 'train')
# dev_img = get_image_vectors(dataset, 'validation')
# test_img = get_image_vectors(dataset, 'test')

# torch.save(train_img, 'train_img_tensors.pt')
# torch.save(dev_img, 'dev_img_tensors.pt')
# torch.save(test_img, 'test_img_tensors.pt')

In [4]:
train_img = torch.load('train_img_tensors.pt')
dev_img = torch.load('dev_img_tensors.pt')
test_img = torch.load('test_img_tensors.pt')

In [5]:
train_X = [np.array(x.cpu()).flatten() for x in train_img]
dev_X = [np.array(x.cpu()).flatten() for x in dev_img]
test_X = [np.array(x.cpu()).flatten() for x in test_img]

In [6]:
Y_train = np.asarray(dataset['train']['label'])
Y_dev = np.asarray(dataset['validation']['label'])
Y_test = np.asarray(dataset['test']['label'])

In [ ]:
from sklearn.svm import SVC
clf_svc = SVC(kernel='rbf', max_iter = 100000) # parameter C was selected based on grid search
clf_svc.fit(train_X, Y_train)
Y_pred = clf_svc.predict(dev_X)
results = pd.DataFrame(
    [list(precision_recall_fscore_support(Y_dev, Y_pred, average='macro')[:3])],
    columns=['precision', 'recall', 'F1'])
results

In [ ]:
Y_pred2 = clf_svc.predict(test_X)

In [ ]:
df_dev = pd.read_csv(dev, keep_default_na=False)
df_test = pd.read_csv(test, keep_default_na=False)

df_dev['ResNet_svm_rbf_kernel'] = Y_pred
df_test['ResNet_svm_rbf_kernel'] = Y_pred_2

df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
df_test.to_csv(datafolder+'test_with_features.csv', index=False)

In [ ]:
# df_dev = pd.read_csv(dev, keep_default_na=False)
# df_test = pd.read_csv(test, keep_default_na=False)

# df_dev['ResNet_svm_rbf_kernel'] = Y_pred
# df_test['ResNet_svm_rbf_kernel'] = Y_pred_2

# df_dev.to_csv(datafolder+'dev_with_features.csv', index=False)
# df_test.to_csv(datafolder+'test_with_features.csv', index=False)